In [1]:
# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

# Важная настройка для корректной настройки pipeline!
import sklearn
sklearn.set_config(transform_output="pandas")

# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, OrdinalEncoder, TargetEncoder, PowerTransformer, LabelEncoder
from sklearn.model_selection import GridSearchCV, KFold

# for model learning
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score

#models
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBRegressor

# Metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score, mean_absolute_error, root_mean_squared_error, root_mean_squared_log_error

# tunning hyperparamters model
import optuna

/home/marena/miniforge3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Загружаем данные 

In [2]:
df = pd.read_csv('data_piped/data.csv', index_col='Id')
target = pd.read_csv('data_piped/target.csv', index_col='Id')

In [3]:
df.head()

,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,Average_Qual,CompAge,ReNew,Baths,Area,HQArea
Id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.113306,-0.310052,-0.066884,-0.19125,0.149281,-1.089162,-0.829479,1.492840,0.512462,0.138784
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.113306,-0.310052,-0.066884,-0.19125,1.462802,0.126900,-0.829479,0.423128,-0.255065,0.085053
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.113306,-0.310052,-0.066884,-0.19125,0.149281,-0.932513,0.037948,1.492840,0.641615,0.313246
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.113306,-0.310052,-0.066884,-0.19125,0.149281,1.421179,1.533202,-0.191229,0.374109,0.018898
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.113306,-0.310052,-0.066884,-0.19125,0.785447,-0.863117,-0.829479,1.492840,1.413078,1.030790


#### Настраиваем отображение

In [4]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

#### Проверяем на пропуски

In [5]:
for cols in df.columns:
    missing = df[cols].isna().sum()
    if missing > 0:
        print(f'{cols}: {missing}')

## Обучаем модели

#### Разделяем выборки для дальнейшего использования

In [6]:
# Разделяем на трейн и тест наш общий обработанный датасет
train_data, test_data = df.iloc[:target.shape[0]], df.iloc[target.shape[0]:]

In [7]:
boot_data = train_data.copy()
boot_data['Target'] = target

In [8]:
boot_data['Target'].isna()

Id
1       False
2       False
3       False
4       False
5       False
        ...  
1456    False
1457    False
1458    False
1459    False
1460    False
Name: Target, Length: 1460, dtype: bool

#### Делаем бутстрап, потому что мы можем

In [9]:
def bootstrap(data, target_col, model, n_samples, random_seed=42):
    models = []
    errors = []

    X = data.drop(columns=[target_col])
    y = data[target_col]

    for i in tqdm(range(n_samples), desc="Бутстрап итерации"):
        bootstrap_sample = data.sample(frac=1, replace=True, random_state=random_seed + i)
        X_boot = bootstrap_sample.drop(columns=[target_col])
        y_boot = bootstrap_sample[target_col]

        model_clone = model.__class__(**model.get_params())
        model_clone.fit(X_boot, y_boot)
        models.append(model_clone)

        y_pred = model_clone.predict(X)
        error = root_mean_squared_log_error(y, y_pred)
        errors.append(error)
        error_min = min(errors)
    
    return models, error_min

In [10]:
def bootstrap_pred(models, data):
    pred = np.array([model.predict(data) for model in models])
    return pred.mean(axis=0)

#### Теперь обучаем модельки на обучающей выборке и потом выводим результат на тестовой

#### RandomForest

In [11]:
model = RandomForestRegressor(random_state=42, n_estimators=250)

models, y_train_boot = bootstrap(boot_data, target_col='Target', model=model, n_samples=20)

print(f'RMSE={y_train_boot}')

Бутстрап итерации:   0%|          | 0/20 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [15]:
final_pred_RFR = bootstrap_pred(models, test_data)
f_rfr = np.exp(final_pred_RFR)

#### CatBoost

In [11]:
model_cat = CatBoostRegressor(verbose=False)

models_cat, RMSE = bootstrap(boot_data, target_col='Target', model=model_cat, n_samples=100)

print(f'RMSE for Cats = {RMSE}')

Бутстрап итерации: 100%|██████████| 100/100 [06:16<00:00,  3.76s/it]

RMSE for Cats = 0.005110357974101573


In [13]:
fin_cat = bootstrap_pred(models_cat, test_data)
f_cat = np.exp(fin_cat)
print(f_cat)

[125155.73096404 162938.44840473 188776.95861899 ... 160198.8653923
 119219.13312349 222948.70401387]


#### XGBRegressor

In [18]:
moda = XGBRegressor(n_estimators=2000, learning_rate=0.001, device="cuda")

models_XGBR, y_train_boot = bootstrap(boot_data, target_col='Target', model=moda, n_samples=20)

print(f'RMS for XGBoost = {y_train_boot}')

Бутстрап итерации:   0%|          | 0/20 [00:00<?, ?it/s]/home/marena/miniforge3/envs/myenv/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:25:30] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1737531586782/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
Бутстрап итерации: 100%|██████████| 20/20 [03:35<00:00, 10.75s/it]

RMS for XGBoost = 0.00911087883954182


In [19]:
fin_xgb = bootstrap_pred(models_XGBR, test_data)
f_xgb = np.exp(fin_xgb)

#### Пробуем предположение

In [20]:
le_fin = f_xgb * 0.1 + f_rfr * 0.1 + f_cat * 0.8

#### Формируем submission и загружаем его на кагл

In [15]:
output = pd.DataFrame({'Id': test_data.index,
                       'SalePrice': f_cat})
output.to_csv('submissions/submission_catboost.csv', index=False)

In [22]:
output.shape

(1459, 2)

In [23]:
test_data

,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_NA,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,...,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,Average_Qual,CompAge,ReNew,Baths,HQArea
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.561176,0.501987,-0.760542,0.425858,-0.501022,-1.138286,-0.805318,0.556615,2.722693,-0.457529,-0.325009,-0.630013,-0.865051,-0.117872,-1.426371,-0.841195,-0.252538,-1.030739,-0.768828,-1.047060,-0.165939,-0.943288,-1.002976,0.147596,-1.017611,1.144651,0.985160,-1.080224,-0.431952,-0.113306,3.225239,-0.066884,-0.191250,-0.445321,0.609578,-0.829479,-1.667618,-1.080113
1462,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.605607,0.927529,-0.036725,0.425858,-0.592658,-1.234028,1.136189,0.977243,-0.367305,-0.096611,0.655467,0.592029,-0.865051,-0.117872,-0.209238,-0.841195,-0.252538,-1.030739,1.292194,0.190094,-0.165939,-0.193464,-1.002976,0.132069,-1.017611,-0.683271,1.298514,0.579247,-0.431952,-0.113306,-0.310052,-0.066884,5.228748,0.149281,0.679044,-0.829479,-0.879007,-0.072668
1463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.289114,0.862236,-0.760542,-0.480136,0.861772,0.589773,-0.805318,0.875900,-0.367305,-0.923751,-0.219236,-0.520017,1.145986,-0.117872,0.416950,-0.841195,-0.252538,0.800571,1.292194,0.190094,-0.165939,-0.193464,0.822328,0.336470,0.281985,0.097632,1.114096,0.552760,-0.431952,-0.113306,-0.310052,-0.066884,-0.191250,-0.997950,-0.572368,0.037948,0.423078,0.240619
1464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.471550,0.192742,-0.036725,0.425858,0.907565,0.589773,0.703011,0.705057,-0.367305,-0.304729,-0.223808,-0.526773,1.139607,-0.117872,0.369422,-0.841195,-0.252538,0.800571,1.292194,0.190094,-0.165939,0.451374,0.822328,0.341784,0.281985,0.044563,1.272843,0.579247,-0.431952,-0.113306,-0.310052,-0.066884,-0.191250,0.149281,-0.624706,-0.829479,0.423078,0.211152
1465,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [24]:
out_xgb = pd.DataFrame({'Id': test_data.index,
                       'SalePrice': f_xgb})
out_xgb.to_csv('submissions/submission_xgb.csv', index=False)
out_xgb.shape

(1459, 2)

In [25]:
out_rfr = pd.DataFrame({'Id': test_data.index,
                       'SalePrice': f_rfr})
out_rfr.to_csv('submissions/submission_rfr.csv', index=False)

In [26]:
out_rfr.shape

(1459, 2)

#### Тестим новый сабмишн


In [27]:
out_test = pd.DataFrame({'Id': test_data.index, 
                       'SalePrice': le_fin})
out_test.to_csv('submissions/submission_test.csv', index=False)

In [28]:
out_test.shape

(1459, 2)